# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import GEOparse as Geo
import seaborn as sns
from pandas.core.ops import comp_method_OBJECT_ARRAY
from collections import Counter

# from sanbomics import annotate
os.getcwd()

In [ ]:
fpkms_df = pd.read_csv("../raw/GSE183984_ASAN_RNASEQ_FPKM_ensg.csv", index_col=0).T
raw_counts_df = pd.read_csv("../raw/GSE183984_ASAN_RNASEQ_raw_counts_ensg.csv", index_col=0).T

print(fpkms_df.shape)
print(raw_counts_df.shape)

# columns are the same in TPM and FPKM
print(set(fpkms_df.columns) - set(raw_counts_df.columns))
print(set(raw_counts_df.columns) - set(fpkms_df.columns))

# no duplicate column names
print(len(fpkms_df.columns), len(set(fpkms_df.columns)))
print(len(raw_counts_df.columns), len(set(raw_counts_df.columns)))

data_ensemble_genes = list(fpkms_df.columns)

# no transcript level distinction
cnt = Counter()
for gene in data_ensemble_genes:
    cnt[gene[3]] += 1
cnt

In [ ]:
tpms_df = fpkms_df.T.div(fpkms_df.sum(axis=1)).T * 10 ** 6
tpms_df

In [ ]:
def parse_gtf(gtf_path):
    ensemble_gene_id_to_length = Counter()
    ensemble_gene_id_to_version = dict()
    ensemble_gene_id_to_hgnc = dict()
    ensemble_gene_id_to_biotype = dict()
    with open(gtf_path, "r") as f:
        for li, line in enumerate(f):
            # print(line)
            if line.startswith("#"):
                continue
            
            parts = line.strip().split()
            chromosome = parts[0]
            entry_source_0 = parts[1]
            entry_type = parts[2]
            start = int(parts[3])
            end = int(parts[4])
            info = dict()
            for i in range(8, len(parts) - 1, 2):
                key = parts[i]
                val = parts[i + 1][1:-2]
                if key == 'gene_name' and val == 'havana':
                    print(key, val, i)
                info[key] = val
            
            if entry_type == 'gene' and 'gene_id' in info and 'gene_name' in info:
                ensemble_gene_id_to_hgnc[info['gene_id']] = info['gene_name']
                if 'gene_biotype' in info:
                    ensemble_gene_id_to_biotype[info['gene_id']] = info['gene_biotype']
            elif entry_type == 'exon':
                version = info['gene_version']
                ensemble_gene_id = info['gene_id']
                ensemble_gene_id_to_version[ensemble_gene_id] = version
                ensemble_gene_id_to_length[ensemble_gene_id] += int(parts[4]) - int(parts[3]) + 1
                    
    return ensemble_gene_id_to_hgnc, ensemble_gene_id_to_length, ensemble_gene_id_to_version, ensemble_gene_id_to_biotype

ensemble_gene_id_to_hgnc, ensemble_gene_id_to_length, \
 ensemble_gene_id_to_version, ensemble_gene_id_to_biotype = parse_gtf('../data/Homo_sapiens.GRCh38.113.gtf')

In [ ]:
print(len(ensemble_gene_id_to_hgnc.keys()))
print(len(ensemble_gene_id_to_biotype))
print(len(data_ensemble_genes))
print(len(set(ensemble_gene_id_to_hgnc.keys()) - set(data_ensemble_genes)))
print(len(set(data_ensemble_genes) - set(ensemble_gene_id_to_hgnc.keys())))

ensemble_genes_common = list(set(data_ensemble_genes).intersection(ensemble_gene_id_to_hgnc.keys()))
len(ensemble_genes_common)

In [ ]:
from collections import Counter

cnt = Counter()
hgnc_to_ensemble_set = dict()
# for key in ensemble_gene_id_to_hgnc:
for key in ensemble_genes_common:
    hgnc = ensemble_gene_id_to_hgnc[key]
    cnt[hgnc] += 1
    if cnt[hgnc] == 1:
        hgnc_to_ensemble_set[hgnc] = {key}
    else:
        hgnc_to_ensemble_set[hgnc].add(key)


In [ ]:
parsed_hgncs = set(hgnc_to_ensemble_set.keys())
bg_hgncs = set(open('../data/gnames.txt').read().strip().split())

print('Excess genes: ', len(parsed_hgncs - bg_hgncs))
print('Missing genes from BG list: ', len(bg_hgncs - parsed_hgncs))

common_hgncs = parsed_hgncs.intersection(bg_hgncs)
print('Common genes: ', len(common_hgncs))

In [ ]:
for key in common_hgncs:
    if len(hgnc_to_ensemble_set[key]) > 1 and key is not None:
        print(key)
        print([(egene, ensemble_gene_id_to_length[egene], ensemble_gene_id_to_biotype[egene], ensemble_gene_id_to_version[egene]) for egene in hgnc_to_ensemble_set[key]])

In [ ]:
def process_columns(df):
    print('Initial df cols: ', len(df.columns))
    df = df.drop(columns=list(set(df.columns) - set(ensemble_gene_id_to_hgnc.keys())))
    print('After dropping cols not annotated with HGNC name in GTF: ', len(df.columns))
    
    renamer = dict()
    for egene in ensemble_gene_id_to_hgnc:
        if ensemble_gene_id_to_biotype[egene] == 'protein_coding':
            renamer[egene] = ensemble_gene_id_to_hgnc[egene]

    df = df.rename(columns=renamer)
    df = df.drop(columns=[col for col in df.columns if col.startswith('ENSG')])
    
    print('After renaming to HGNC (and deleting non-protein coding egenes): ', len(df.columns))
    
    df = df[list(common_hgncs.intersection(df.columns))]

    print('After shrinking to BG HGNCs that are in the table: ', len(df.columns))
    
    mask = df.columns.duplicated()
    for i in range(len(mask)):
        if mask[i]:
            j = 0
            while df.columns[j] != df.columns[i]:
                j += 1
            df[df.columns[j]] += df[df.columns[i]]

    df = df.loc[:,~df.columns.duplicated()]

    print('After summing columns with the same HGNC name: ', len(df.columns))

    return df

raw_counts_df = process_columns(raw_counts_df)
fpkms_df = process_columns(fpkms_df)
tpms_df = process_columns(tpms_df)

In [ ]:
tpms_df

In [ ]:
log_tpms_df = tpms_df.apply(lambda x: np.log(1 + x))
plt.figure(figsize=(16, 9))
sns.histplot(log_tpms_df.iloc[1])
plt.ylim(0, 1000)

In [ ]:
plt.figure(figsize=(16, 9))
sns.histplot(log_tpms_df.mean())
plt.ylim(0, 1000)